In [56]:
!pip install librosa numpy pandas matplotlib tensorflow-macos tensorflow-metal

  Using cached tensorflow_macos-2.16.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached tensorflow_metal-1.2.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (1.3 kB)
  Using cached tensorflow-2.16.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_macos-2.16.2-cp312-cp312-macosx_12_0_arm64.whl (2.1 kB)
Using cached tensorflow-2.16.2-cp312-cp312-macosx_12_0_arm64.whl (227.1 MB)
Using cached tensorflow_metal-1.2.0-cp312-cp312-macosx_12_0_arm64.whl (1.4 MB)
Using cached ml_dtypes-0.3.2-cp312-cp312-macosx_10_9_universal2.whl (393 kB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Foun

In [57]:
import os
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, ReLU, Dropout, Dense, Lambda
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

# Define paths
dataset_path = "/Users/aaryan/Downloads/6967442"  
train_metadata_path = os.path.join(dataset_path, "metadata of train set .csv")
test_metadata_path = os.path.join(dataset_path, "metadata of test set.csv")
train_folder = os.path.join(dataset_path, "train")
test_folder = os.path.join(dataset_path, "test")

# Load metadata
train_metadata = pd.read_csv(train_metadata_path)
test_metadata = pd.read_csv(test_metadata_path)

In [58]:
# Audio parameters
SAMPLE_RATE = 8000
AUDIO_LENGTH = 4 * SAMPLE_RATE  # 4 seconds * 8000 Hz
NUM_CLASSES = train_metadata['Class ID'].nunique()
def load_training_audio(filename):
    """Load and preprocess audio file"""
    filepath = os.path.join(train_folder, filename)
    signal, _ = librosa.load(filepath, sr=SAMPLE_RATE)
    
    # Zero-pad if shorter than required length
    if len(signal) < AUDIO_LENGTH:
        pad_length = AUDIO_LENGTH - len(signal)
        signal = np.pad(signal, (0, pad_length))
    else:
        signal = signal[:AUDIO_LENGTH]  # Truncate if longer
    
    return signal.reshape(-1, 1)  # Reshape for Conv1D

def load_testing_audio(filename):
    """Load and preprocess audio file"""
    filepath = os.path.join(test_folder, filename)
    signal, _ = librosa.load(filepath, sr=SAMPLE_RATE)
    
    # Zero-pad if shorter than required length
    if len(signal) < AUDIO_LENGTH:
        pad_length = AUDIO_LENGTH - len(signal)
        signal = np.pad(signal, (0, pad_length))
    else:
        signal = signal[:AUDIO_LENGTH]  # Truncate if longer
    
    return signal.reshape(-1, 1)  # Reshape for Conv1D


# Prepare dataset
def prepare_train_data(metadata):
    X, y = [], []
    for _, row in metadata.iterrows():
        audio = load_training_audio(row['Filename'])
        X.append(audio)
        y.append(row['Class ID'])
    return np.array(X), np.array(y)

def prepare_test_data(metadata):
    X, y = [], []
    for _, row in metadata.iterrows():
        audio = load_testing_audio(row['Filename'])
        X.append(audio)
        y.append(row['Class_id'])
    return np.array(X), np.array(y)

X_train, y_train = prepare_train_data(train_metadata)
X_test, y_test = prepare_test_data(test_metadata)




In [59]:
if tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    print("Using MPS backend for training")

Using MPS backend for training


In [60]:
# Train-validation split (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42, stratify=y_train)

# Define model
def build_model():
    model = Sequential([
        Conv1D(64, kernel_size=80, strides=4, padding='same', activation=None, input_shape=(32000, 1)),
        BatchNormalization(), ReLU(),
        MaxPooling1D(pool_size=4), Dropout(0.1),

        Conv1D(64, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Conv1D(64, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        MaxPooling1D(pool_size=4), Dropout(0.1),

        Conv1D(128, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Conv1D(128, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        MaxPooling1D(pool_size=4), Dropout(0.1),

        Conv1D(256, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Conv1D(256, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Conv1D(256, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        MaxPooling1D(pool_size=4), Dropout(0.1),

        Conv1D(512, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Conv1D(512, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Dropout(0.1),

        Conv1D(512, kernel_size=3, strides=1, padding='same', activation=None),
        BatchNormalization(), ReLU(),
        Lambda(lambda x: tf.reduce_mean(x, axis=1)),  # Global average pooling
        Dropout(0.2),

        Dense(7, activation='softmax')  # 7 classes
    ])
    
    return model

# Compile model
model = build_model()
model.compile(optimizer=SGD(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callback to save the best model based on validation loss
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True, mode="min", verbose=1)

# Train model with callbacks
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_val, y_val), verbose=1, callbacks=[checkpoint])

# Load best model
model.load_weights("best_model.h5")



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.3365 - loss: 1.7306
Epoch 1: val_loss improved from inf to 1.92385, saving model to best_model.h5


158/158 ━━━━━━━━━━━━━━━━━━━━ 36s 203ms/step - accuracy: 0.3369 - loss: 1.7298 - val_accuracy: 0.2687 - val_loss: 1.9239
Epoch 2/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.4769 - loss: 1.4017
Epoch 2: val_loss did not improve from 1.92385
158/158 ━━━━━━━━━━━━━━━━━━━━ 29s 184ms/step - accuracy: 0.4770 - loss: 1.4016 - val_accuracy: 0.2719 - val_loss: 2.1204
Epoch 3/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.5232 - loss: 1.2826
Epoch 3: val_loss improved from 1.92385 to 1.82287, saving model to best_model.h5


158/158 ━━━━━━━━━━━━━━━━━━━━ 29s 184ms/step - accuracy: 0.5231 - loss: 1.2827 - val_accuracy: 0.3076 - val_loss: 1.8229
Epoch 4/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.5432 - loss: 1.2098
Epoch 4: val_loss improved from 1.82287 to 1.38182, saving model to best_model.h5


158/158 ━━━━━━━━━━━━━━━━━━━━ 29s 184ms/step - accuracy: 0.5432 - loss: 1.2098 - val_accuracy: 0.4714 - val_loss: 1.3818
Epoch 5/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.5673 - loss: 1.1490
Epoch 5: val_loss improved from 1.38182 to 1.18515, saving model to best_model.h5


...

158/158 ━━━━━━━━━━━━━━━━━━━━ 31s 197ms/step - accuracy: 0.9067 - loss: 0.2691 - val_accuracy: 0.8593 - val_loss: 0.4874
Epoch 194/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.9155 - loss: 0.2509
Epoch 194: val_loss did not improve from 0.48740
158/158 ━━━━━━━━━━━━━━━━━━━━ 33s 211ms/step - accuracy: 0.9155 - loss: 0.2509 - val_accuracy: 0.8450 - val_loss: 0.5770
Epoch 195/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.9141 - loss: 0.2508
Epoch 195: val_loss did not improve from 0.48740
158/158 ━━━━━━━━━━━━━━━━━━━━ 31s 199ms/step - accuracy: 0.9141 - loss: 0.2508 - val_accuracy: 0.8331 - val_loss: 0.6274
Epoch 196/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.9156 - loss: 0.2586
Epoch 196: val_loss did not improve from 0.48740
158/158 ━━━━━━━━━━━━━━━━━━━━ 31s 196ms/step - accuracy: 0.9156 - loss: 0.2585 - val_accuracy: 0.8052 - val_loss: 0.7936
Epoch 197/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.9207 - loss: 0.2427
Epoch 197

158/158 ━━━━━━━━━━━━━━━━━━━━ 34s 214ms/step - accuracy: 0.9319 - loss: 0.2158 - val_accuracy: 0.8601 - val_loss: 0.4760
Epoch 219/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.9252 - loss: 0.2338
Epoch 219: val_loss did not improve from 0.47597
158/158 ━━━━━━━━━━━━━━━━━━━━ 34s 212ms/step - accuracy: 0.9252 - loss: 0.2338 - val_accuracy: 0.7893 - val_loss: 1.0148
Epoch 220/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.9297 - loss: 0.2305
Epoch 220: val_loss did not improve from 0.47597
158/158 ━━━━━━━━━━━━━━━━━━━━ 32s 202ms/step - accuracy: 0.9297 - loss: 0.2304 - val_accuracy: 0.8521 - val_loss: 0.5348
Epoch 221/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.9212 - loss: 0.2165
Epoch 221: val_loss did not improve from 0.47597
158/158 ━━━━━━━━━━━━━━━━━━━━ 33s 206ms/step - accuracy: 0.9213 - loss: 0.2165 - val_accuracy: 0.8498 - val_loss: 0.5642
Epoch 222/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.9189 - loss: 0.2382
Epoch 222

158/158 ━━━━━━━━━━━━━━━━━━━━ 34s 212ms/step - accuracy: 0.9293 - loss: 0.2225 - val_accuracy: 0.8577 - val_loss: 0.4609
Epoch 225/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.9310 - loss: 0.2185
Epoch 225: val_loss did not improve from 0.46089
158/158 ━━━━━━━━━━━━━━━━━━━━ 33s 210ms/step - accuracy: 0.9310 - loss: 0.2185 - val_accuracy: 0.8219 - val_loss: 0.7729
Epoch 226/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.9292 - loss: 0.2099
Epoch 226: val_loss did not improve from 0.46089
158/158 ━━━━━━━━━━━━━━━━━━━━ 34s 212ms/step - accuracy: 0.9293 - loss: 0.2099 - val_accuracy: 0.8585 - val_loss: 0.4972
Epoch 227/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.9273 - loss: 0.2185
Epoch 227: val_loss did not improve from 0.46089
158/158 ━━━━━━━━━━━━━━━━━━━━ 31s 198ms/step - accuracy: 0.9273 - loss: 0.2185 - val_accuracy: 0.8339 - val_loss: 0.6276
Epoch 228/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.9334 - loss: 0.2158
Epoch 228

158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 191ms/step - accuracy: 0.9620 - loss: 0.1243 - val_accuracy: 0.8744 - val_loss: 0.4411
Epoch 353/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.9558 - loss: 0.1320
Epoch 353: val_loss did not improve from 0.44107
158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 191ms/step - accuracy: 0.9558 - loss: 0.1320 - val_accuracy: 0.8037 - val_loss: 1.0333
Epoch 354/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9564 - loss: 0.1402
Epoch 354: val_loss did not improve from 0.44107
158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 189ms/step - accuracy: 0.9564 - loss: 0.1401 - val_accuracy: 0.8394 - val_loss: 0.6975
Epoch 355/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9582 - loss: 0.1287
Epoch 355: val_loss did not improve from 0.44107
158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 189ms/step - accuracy: 0.9582 - loss: 0.1287 - val_accuracy: 0.8625 - val_loss: 0.5874
Epoch 356/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9534 - loss: 0.1397
Epoch 356

158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 192ms/step - accuracy: 0.9626 - loss: 0.1267 - val_accuracy: 0.8855 - val_loss: 0.4010
Epoch 370/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.9641 - loss: 0.1209
Epoch 370: val_loss did not improve from 0.40098
158/158 ━━━━━━━━━━━━━━━━━━━━ 31s 197ms/step - accuracy: 0.9641 - loss: 0.1210 - val_accuracy: 0.8736 - val_loss: 0.4834
Epoch 371/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.9533 - loss: 0.1385
Epoch 371: val_loss did not improve from 0.40098
158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 192ms/step - accuracy: 0.9533 - loss: 0.1385 - val_accuracy: 0.8410 - val_loss: 0.7056
Epoch 372/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.9588 - loss: 0.1293
Epoch 372: val_loss did not improve from 0.40098
158/158 ━━━━━━━━━━━━━━━━━━━━ 30s 191ms/step - accuracy: 0.9588 - loss: 0.1293 - val_accuracy: 0.8180 - val_loss: 0.9367
Epoch 373/500
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.9620 - loss: 0.1220
Epoch 373

In [63]:
# Evaluate model
def evaluate_model(model, X, y, dataset_name="Dataset"):
    y_pred = np.argmax(model.predict(X), axis=1)
    print(f"\nClassification Report for {dataset_name}:")
    print(classification_report(y, y_pred, digits=4))

# Report metrics
evaluate_model(model, X_train, y_train, "Training Set")
evaluate_model(model, X_val, y_val, "Validation Set")
evaluate_model(model, X_test, y_test, "Test Set")

158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step

Classification Report for Training Set:
              precision    recall  f1-score   support

           0     0.9861    0.9956    0.9908      1352
           1     0.9882    0.9921    0.9901       506
           2     0.9922    0.9742    0.9831      1433
           3     0.9770    0.9834    0.9802       906
           4     0.9957    0.9852    0.9905       474
           5     0.9787    0.9735    0.9761       189
           6     0.9066    0.9649    0.9348       171

    accuracy                         0.9841      5031
   macro avg     0.9749    0.9813    0.9780      5031
weighted avg     0.9843    0.9841    0.9841      5031

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step

Classification Report for Validation Set:
              precision    recall  f1-score   support

           0     0.9298    0.9793    0.9539       338
           1     0.8803    0.8175    0.8477       126
           2     0.9008    0.8883    0.8945       358
           3     0.8